In [ ]:
!pip install transformers
# !pip install wandb
# !pip install pytorch-pretrained-bert==0.4.0
!pip install --upgrade tqdm

     |████████████████████████████████| 3.4 MB 5.0 MB/s 
     |████████████████████████████████| 895 kB 60.6 MB/s 
     |████████████████████████████████| 3.3 MB 56.1 MB/s 
     |████████████████████████████████| 596 kB 72.9 MB/s 
     |████████████████████████████████| 67 kB 6.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
torch.cuda.empty_cache()

# Setup the data set

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
unique_labels = set(['X', '[CLS]', '[SEP]'])

import six

# def convert_to_unicode(text):
#   """Converts `text` to Unicode (if it's not already), assuming utf-8 input."""
#   if six.PY3:
#     if isinstance(text, str):
#       return text
#     elif isinstance(text, bytes):
#       return text.decode("utf-8", "ignore")
#     else:
#       raise ValueError("Unsupported string type: %s" % (type(text)))
#   elif six.PY2:
#     if isinstance(text, str):
#       return text.decode("utf-8", "ignore")
#     elif isinstance(text, unicode):
#       return text
#     else:
#       raise ValueError("Unsupported string type: %s" % (type(text)))
#   else:
#     raise ValueError("Not running on Python2 or Python 3?")


class InputExample(object):
  """A single training/test example for simple sequence classification."""

  def __init__(self, guid, text, label=None, segment_ids=None):
    """Constructs a InputExample.
    Args:
      guid: Unique id for the example.
      text_a: string. The untokenized text of the first sequence. For single
        sequence tasks, only this sequence must be specified.
      label: (Optional) string. The label of the example. This should be
        specified for train and dev examples, but not for test examples.
    """
    self.guid = guid
    self.text = text
    self.label = label
    self.segment_ids = segment_ids

# class DataProcessor(object):
#     """Base class for data converters for sequence classification data sets."""

#     def get_train_examples(self, data_dir):
#         """Gets a collection of `InputExample`s for the train set."""
#         raise NotImplementedError()

#     def get_dev_examples(self, data_dir):
#         """Gets a collection of `InputExample`s for the dev set."""
#         raise NotImplementedError()

#     def get_labels(self):
#         """Gets the list of labels for this data set."""
#         raise NotImplementedError()

#     @classmethod
#     def _read_data(cls,input_file):
#         """Read a BIO data!"""
#         rf = open(input_file,'r')
#         lines = [];words = [];labels = []
#         for line in rf:
#             if line.strip().startswith('-DOCSTART-'):
#               continue
#             word = line.strip().split(' ')[0]
#             label = line.strip().split(' ')[-1]
#             # here we dont do "DOCSTART" check
#             try:
#               if len(line.strip())==0 and words[-1] == '.':
#                   l = ' '.join([label for label in labels if len(label) > 0])
#                   w = ' '.join([word for word in words if len(word) > 0])
#                   lines.append((l,w))
#                   words = []
#                   labels = []
#             except IndexError:
#               print(len(word))
#               continue
#             words.append(word)
#             labels.append(label)
#             if len(label) > 0:
#               unique_labels.add(label)
#         rf.close()
#         return lines

# class NerProcessor(DataProcessor):
#     def get_train_examples(self, data_dir):
#         return self._create_example(
#             self._read_data(os.path.join(data_dir, "/content/drive/My Drive/bert-ner/train.txt")), "train"
#         )

#     def get_dev_examples(self, data_dir):
#         return self._create_example(
#             self._read_data(os.path.join(data_dir, "/content/drive/My Drive/bert-ner/valid.txt")), "dev"
#         )

#     def get_test_examples(self,data_dir):
#         return self._create_example(
#             self._read_data(os.path.join(data_dir, "/content/drive/My Drive/bert-ner/test.txt")), "test"
#         )


#     def get_labels(self):
#         """
#         here "X" used to represent "##eer","##soo" and so on!
#         "[PAD]" for padding
#         :return:
#         """
#         return list(unique_labels)

#     def _create_example(self, lines, set_type):
#         examples = []
#         for (i, line) in enumerate(lines):
#             guid = "%s-%s" % (set_type, i)
#             texts = convert_to_unicode(line[1])
#             labels = convert_to_unicode(line[0])
#             examples.append(InputExample(guid=guid, text=texts, label=labels))
#         return examples

In [ ]:
def readfile(filename):
    '''
    read file
    '''
    f = open(filename)
    data = []
    sentence = []
    label= []
    for line in f:
        if len(line)==0 or line.startswith('-DOCSTART') or line[0]=="\n":
            if len(sentence) > 0:
                data.append((sentence,label))
                sentence = []
                label = []
            continue
        splits = line.split(' ')
        sentence.append(splits[0])
        label.append(splits[-1][:-1])

    if len(sentence) >0:
        data.append((sentence,label))
        sentence = []
        label = []
    return data

class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        return readfile(input_file)


class NerProcessor(DataProcessor):
    """Processor for the CoNLL-2003 data set."""

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "/content/drive/My Drive/train.txt")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "/content/drive/My Drive/valid.txt")), "dev")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "/content/drive/My Drive/test.txt")), "test")

    def get_labels(self):
        return ["O", "B-MISC", "I-MISC",  "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "[CLS]", "[SEP]", "X"]

    def _create_examples(self,lines,set_type):
        examples = []
        for i,(sentence,label) in enumerate(lines):
            guid = "%s-%s" % (set_type, i)
            text_a = ' '.join(sentence)
            label = label
            examples.append(InputExample(guid=guid,text=text_a,label=label))
        return examples

In [ ]:
import os
train_examples = NerProcessor().get_train_examples('.')
val_examples = NerProcessor().get_dev_examples('.')
test_examples = NerProcessor().get_test_examples('.')

In [ ]:
print(len(train_examples))
print(len(val_examples))
print(len(test_examples))

14041
3250
3453


In [ ]:
train_examples[0].label

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [ ]:
tags_vals = NerProcessor().get_labels()
label_map = {}
#here start with zero this means that "[PAD]" is zero
for (i,label) in enumerate(tags_vals):
    label_map[label] = i

In [ ]:
pip install pytorch_pretrained_bert==0.4.0

     |████████████████████████████████| 45 kB 1.7 MB/s 
     |████████████████████████████████| 131 kB 8.8 MB/s 
     |████████████████████████████████| 79 kB 9.8 MB/s 
     |████████████████████████████████| 8.5 MB 62.4 MB/s 
     |████████████████████████████████| 138 kB 58.7 MB/s 
     |████████████████████████████████| 127 kB 71.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
label_map

{'B-LOC': 7,
 'B-MISC': 1,
 'B-ORG': 5,
 'B-PER': 3,
 'I-LOC': 8,
 'I-MISC': 2,
 'I-ORG': 6,
 'I-PER': 4,
 'O': 0,
 'X': 11,
 '[CLS]': 9,
 '[SEP]': 10}

In [ ]:
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig

from transformers import BertForTokenClassification, AdamW
#import transformers.WarmupLinearSchedule

In [ ]:
MAX_LEN = 128
bs = 32

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [ ]:
torch.cuda.get_device_name(0)

'Tesla T4'

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
def make_feature(input_example):
  text = input_example.text
  label = input_example.label
  word_tokens = ['[CLS]']
  label_list = ['[CLS]']
  label_mask = [0]

  input_ids = [tokenizer.convert_tokens_to_ids('[CLS]')]
  label_ids = [label_map['[CLS]']]

  # iterate over individual tokens and their labels
  for word, label in zip(text.split(), label):
    tokenized_word = tokenizer.tokenize(word)

    for token in tokenized_word:
      word_tokens.append(token)
      input_ids.append(tokenizer.convert_tokens_to_ids(token))

    label_list.append(label)
    label_ids.append(label_map[label])
    label_mask.append(1)
    # len(tokenized_word) > 1 only if it splits word in between, in which case
    # the first token gets assigned NER tag and the remaining ones get assigned
    # X
    for i in range(1, len(tokenized_word)):
      label_list.append('X')
      label_ids.append(label_map['X'])
      label_mask.append(0)

  assert len(word_tokens) == len(label_list) == len(input_ids) == len(label_ids) == len(label_mask)

  if len(word_tokens) >= MAX_LEN:
    word_tokens = word_tokens[:(MAX_LEN - 1)]
    label_list = label_list[:(MAX_LEN - 1)]
    input_ids = input_ids[:(MAX_LEN - 1)]
    label_ids = label_ids[:(MAX_LEN - 1)]
    label_mask = label_mask[:(MAX_LEN - 1)]

  assert len(word_tokens) < MAX_LEN, len(word_tokens)

  word_tokens.append('[SEP]')
  label_list.append('[SEP]')
  input_ids.append(tokenizer.convert_tokens_to_ids('[SEP]'))
  label_ids.append(label_map['[SEP]'])
  label_mask.append(0)

  assert len(word_tokens) == len(label_list) == len(input_ids) == len(label_ids) == len(label_mask)

  sentence_id = [0 for _ in input_ids]
  attention_mask = [1 for _ in input_ids]

  while len(input_ids) < MAX_LEN:
    input_ids.append(0)
    label_ids.append(label_map['X'])
    attention_mask.append(0)
    sentence_id.append(0)
    label_mask.append(0)

  assert len(word_tokens) == len(label_list)
  assert len(input_ids) == len(label_ids) == len(attention_mask) == len(sentence_id) == len(label_mask) == MAX_LEN, len(input_ids)
  return word_tokens, label_list, input_ids, label_ids, attention_mask, sentence_id, label_mask

In [ ]:
import numpy as np

def setup_data():
  data = {}
  for ds in [('train', train_examples), ('val', val_examples), ('test', test_examples)]:
    input_ids = []
    tags = []
    attention_masks = []
    token_type_ids = []
    raw_text = []
    raw_labels = []
    label_masks = []
    for example in ds[1]:
      word_tokens, label_list, input_id, label_id, attention_mask, sentence_id, label_mask = make_feature(example)
      raw_text.append(word_tokens)
      raw_labels.append(label_list)
      input_ids.append(input_id)
      tags.append(label_id)
      attention_masks.append(attention_mask)
      token_type_ids.append(sentence_id)
      label_masks.append(label_mask)

    data[ds[0]] = {'inputs':torch.LongTensor(input_ids), 'labels':torch.LongTensor(tags), 'mask':torch.LongTensor(attention_masks), 'raw_text':raw_text, 'raw_labels':raw_labels, 'token_type_ids':torch.LongTensor(token_type_ids), 'label_masks':torch.BoolTensor(label_masks)}

  return data

In [ ]:
dataset = setup_data()

In [ ]:
print("text",dataset['train']['raw_text'][0])
print("label", dataset['train']['raw_labels'][0])
print(dataset['train']['inputs'][0])
print(dataset['train']['labels'][0])
print(dataset['train']['mask'][0])
print(dataset['train']['label_masks'][0])

text ['[CLS]', 'EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'la', '##mb', '.', '[SEP]']
label ['[CLS]', 'B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'X', 'O', '[SEP]']
tensor([  101,  7270, 22961,  1528,  1840,  1106, 21423,  1418,  2495, 12913,
          119,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,    

# Setup the model

In [ ]:
train_data = TensorDataset(dataset['train']['inputs'], dataset['train']['mask'], dataset['train']['labels'], dataset['train']['token_type_ids'], dataset['train']['label_masks'])
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(dataset['val']['inputs'], dataset['val']['mask'], dataset['val']['labels'], dataset['val']['token_type_ids'], dataset['val']['label_masks'])
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [ ]:
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.nn import CrossEntropyLoss

class CoNLLClassifier(BertForTokenClassification):
    def forward(self, input_ids, attention_mask=None, token_type_ids=None,
                position_ids=None, head_mask=None, labels=None, label_masks=None):
        outputs = self.bert(input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                            position_ids=position_ids,
                            head_mask=head_mask)

        sequence_output = outputs[0] #(b, MAX_LEN, 768)
        token_reprs = [embedding[mask] for mask, embedding in zip(label_masks, sequence_output)]
        token_reprs = pad_sequence(sequences=token_reprs, batch_first=True, padding_value=-1) # (b, local_max_len, 768)
        sequence_output = self.dropout(token_reprs)
        logits = self.classifier(sequence_output) # (b, local_max_len, num_labels)
        outputs = (logits,)
        if labels is not None:
            # invalid_label_value = label_map['X']
            labels = [label[mask] for mask, label in zip(label_masks, labels)]
            labels = pad_sequence(labels, batch_first=True, padding_value=-1) #(b, local_max_len)
            loss_fct = CrossEntropyLoss(ignore_index=-1, reduction='sum')
            mask = labels != -1
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            loss /= mask.float().sum()
            outputs = (loss,) + outputs + (labels, )

        return outputs  # (loss), scores, (hidden_states), (attentions)

In [ ]:
model =  .from_pretrained("bert-base-cased", num_labels=len(label_map))

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing CoNLLClassifier: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing CoNLLClassifier from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CoNLLClassifier from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CoNLLClassifier were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.b

In [ ]:
model = model.cuda()

In [ ]:
 #batch = train_data[:20]
 #batch = tuple(t.to('cpu') for t in batch)

In [ ]:
##b_input_ids, b_input_mask, b_labels, b_token_type_ids, b_label_masks = batch
#loss, output, labels = model(b_input_ids, token_type_ids=b_token_type_ids, attention_mask=b_input_mask, labels=b_labels, label_masks=b_label_masks)

In [ ]:
# print(loss)
# print(output.size())
# print(labels.size())

In [ ]:
!pip install pytorch_transformers


     |████████████████████████████████| 176 kB 5.2 MB/s 
     |████████████████████████████████| 1.2 MB 9.3 MB/s 


In [ ]:
num_epochs = 5
from pytorch_transformers.optimization import WarmupLinearSchedule
num_train_optimization_steps = int(len(train_examples)/bs) * num_epochs
FULL_FINETUNING = True
lr = 3e-5
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

warmup_steps = int(0.1 * num_train_optimization_steps)
optimizer = AdamW(optimizer_grouped_parameters, lr=lr)
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_steps, t_total=num_train_optimization_steps)

In [ ]:
!pip install seqeval
from seqeval.metrics import f1_score, accuracy_score, classification_report

     |████████████████████████████████| 43 kB 1.2 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=657a296361606979bde8617f7b8b2379da7ddacf4ef443a7deeddabd206b07c9
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


# Train!

In [ ]:
from tqdm import tqdm_notebook ,tnrange, trange, tqdm
import torch.nn.functional as F
max_grad_norm = 1.0
import numpy as np

for _ in tnrange(num_epochs, desc="Epoch"):
    # TRAIN loop
    model = model.train()
    tr_loss = 0
    nb_tr_steps = 0
    for step, batch in enumerate(tqdm_notebook(train_dataloader)):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels, b_token_type_ids, b_label_masks = batch
        # forward pass
        loss, logits, labels = model(b_input_ids, token_type_ids=b_token_type_ids,
                     attention_mask=b_input_mask, labels=b_labels, label_masks=b_label_masks)
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        scheduler.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    model = model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in tqdm_notebook(valid_dataloader):
        batch = tuple(t.to(device) for t in batch)

        b_input_ids, b_input_mask, b_labels, b_token_type_ids, b_label_masks = batch

        with torch.no_grad():
          tmp_eval_loss, logits, reduced_labels = model(b_input_ids, token_type_ids=b_token_type_ids,
                                  attention_mask=b_input_mask, labels=b_labels, label_masks=b_label_masks)

        logits = torch.argmax(F.log_softmax(logits,dim=2),dim=2)
        logits = logits.detach().cpu().numpy()
        reduced_labels = reduced_labels.to('cpu').numpy()

        labels_to_append = []
        predictions_to_append = []

        for prediction, label in zip(logits, reduced_labels):
          preds = []
          labels = []
          for pred, lab in zip(prediction, label):
            if lab.item() == -1:
              continue
            preds.append(pred)
            labels.append(lab)
          predictions_to_append.append(preds)
          labels_to_append.append(labels)

        predictions.extend(predictions_to_append)
        true_labels.append(labels_to_append)
        # print(logits.shape, label_ids.shape, b_input_mask.size())

        eval_loss += tmp_eval_loss.mean().item()

        nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
    valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


  0%|          | 0/439 [00:00<?, ?it/s]

Train loss: 0.3824266858615304


...

Validation loss: 0.03636525976203391


  0%|          | 0/439 [00:00<?, ?it/s]

Train loss: 0.0060070665510252525


  0%|          | 0/102 [00:00<?, ?it/s]

Validation loss: 0.03593651417060185


In [ ]:
# torch.save(model.state_dict(), '/content/drive/My Drive/bert-ner/model.torch')

In [ ]:
from sklearn.metrics import  recall_score
print("Seq eval accuracy: {}".format(accuracy_score(valid_tags, pred_tags)))
#print("F1-Score: {}".format(f1_score(valid_tags, [pred_tags])))
#print("Classification report: -- ")
print(valid_tags)
print([pred_tags])

print(classification_report([valid_tags], [pred_tags]))

Seq eval accuracy: 0.9917385967305107
['O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'B-MISC', 'I-MISC', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'B-ORG', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', '